In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.cluster import KMeans


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

    
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


# Qual o tipo de aplicativo seria mais rentavel?

## Objetivo:

***1- Identificar quais categorias de aplicativos são mais buscados pelos usuario.***

***2- Identificar qual o indice de satisfação nas categorias listadas .***


 > A analise será baseada em atributos listados de uma base de dados coledata em dezembro de 2020 disponivel no Kaggle. Sendo estes atributos a linguagem do app, classificação, categoria, valor comercializado, avaliação média, reações dos usuarios e/ou numero de instalações. Com esses parametros esperamos identificar o potencial de crescimento dos apps e a satisfação/lealdade do cliente em relaçao a categoria.  

*  O potencial de crecimento será medido através da quantidade de reações/intalações observadas.
*  A satisfação/lealdade será medida através da avaliação média dos aplicativos.
*  O potencial de retorno será avaliado comparando a quantidade de instalações e suas formas de pagamento
* A potencial oportunidade de mercado será levantada cruzando o indice elevado de crescimento e baixa satisfação do usuário.  

  No final será apresentado as caracteristicas mais promissoras para o lançamento de aplicativos diantes do mercado atual, determidado por indicadores indiretos.
  

# Processamento/Tratamento dos dados

Nessa primeira etapa será verificar a existencia de dados duplicados,faltantes,formatados de forma não convencional, dados inválidos e inconscistentes. Caso seja encontrados será feito o devido tratamento para cada caso observado.

In [ ]:
dados=pd.read_csv('../input/google-playstore-apps/Google-Playstore.csv')

In [ ]:
dados.columns


## Analisando os nomes das colunas

> Olhando para coluna, conseguimos notar que alguns nomes possuem espassos o que pode apresentar inconsistencia na execução de alguns comando. Para mitigar esse problema vamos trocar os espassos no meio do nome por "_".


In [ ]:
dados.rename(columns={'App Name':'App_Name', 'App Id':'App_Id', 'Rating Count':'Rating_Count',
       'Minimum Installs':'Minimum_Installs', 'Maximum Installs':'Maximum_Installs',
       'Minimum Android':'Minimum_Android', 'Developer Id':'Developer_Id', 'Developer Website':'Developer_Website',
       'Developer Email':'Developer_Email', 'Last Updated':'Last_Updated', 'Content Rating':'Content_Rating',
       'Privacy Policy':'Privacy_Policy', 'Ad Supported':'Ad_Supported', 'In App Purchases':'In_App_Purchases', 'Editors Choice':'Editors_Choice'},inplace=True)

In [ ]:
dados.head()

In [ ]:
dados.describe()

In [ ]:
dados.info()

In [ ]:
dados.isnull().sum()

## Analisando valores vazies

Conseguimos identificar numeros nulos nas seguintes colunas 
>App_Name, Category, Rating, Rating_Count,Installs, Mimum_Installs, Currency, Minimum_Android, Developer_Id, Developer_Website, Developer_Email, Released, Privacy_Policy.

Porem em nossa abordagem iremos tratar apenas as colunas relevantes para nossas análises e descartaremos as demais.

>Indicadores relevantes:
*   Reações/intalações
*   Avaliação média dos aplicativos
*   Formas de pagamento


In [ ]:
dados.drop(['App_Id','Developer_Id', 'Developer_Website', 'Developer_Email','Privacy_Policy'],inplace=True,axis=1)

In [ ]:
dados.isnull().sum()

In [ ]:
dados.isna().sum()

In [ ]:
dados.Rating.fillna(0,inplace=True)

In [ ]:
dados.Rating_Count.fillna(0,inplace=True)

In [ ]:
dados.isna().sum()

In [ ]:
dados.dropna(axis=0,inplace=True,subset=['Installs','Minimum_Installs','Category'])

In [ ]:
dados.isna().sum()

In [ ]:
dados.info()

In [ ]:
dados.Minimum_Android.unique()

In [ ]:
dados[dados.Minimum_Android.isna()]

In [ ]:
dados.Minimum_Android.fillna('Não Consta', inplace=True)

In [ ]:
dados.Released.fillna('Não Consta',inplace=True)

In [ ]:
dados[dados.App_Name.isna()]

In [ ]:
dados.App_Name.fillna('Não Consta',inplace=True)

In [ ]:
dados[dados.App_Name.isna()]

In [ ]:
dados.isna().sum()

In [ ]:
dados[dados.duplicated()]

In [ ]:
dados.drop_duplicates(inplace=True)

In [ ]:
dados[dados.duplicated()]

# Tratando outliers

In [ ]:
Box_Rating=dados.boxplot(column='Rating', figsize=(12,7))

In [ ]:
dados.Rating.describe()

In [ ]:
Box_Minimum_Installs=dados.boxplot(column='Minimum_Installs', figsize=(12,7))

In [ ]:
valor0=dados.Minimum_Installs

In [ ]:
Q1= valor0.quantile(.25)
Q3= valor0.quantile(.75)
IIQ= Q3-Q1
limite_inferior = Q1 - (1.5*IIQ)
limite_superior = Q3 + (1.5*IIQ)

In [ ]:
selecao = (valor0 >= limite_inferior) & (valor0 <= limite_superior)
dados_Minimum_Installs = dados[selecao]

In [ ]:
Box_Minimum_Installs = dados_Minimum_Installs.boxplot(column='Minimum_Installs', figsize=(12,7))

In [ ]:
Box_Maximum_Installs=dados.boxplot(column='Maximum_Installs', figsize=(12,7))

In [ ]:
valor1=dados.Maximum_Installs

In [ ]:
Q1= valor1.quantile(.25)
Q3= valor1.quantile(.75)
IIQ= Q3-Q1
limite_inferior = Q1 - (1.5*IIQ)
limite_superior = Q3 + (1.5*IIQ)

In [ ]:
selecao = (valor1 >= limite_inferior) & (valor1 <= limite_superior)
dados_Maximum_Installs = dados[selecao]

In [ ]:
Box_Maximum_Installs = dados_Maximum_Installs.boxplot(column='Maximum_Installs', figsize=(12,7))

In [ ]:
dados.boxplot('Price', figsize=(12,7))

In [ ]:
valor2=dados.Price

In [ ]:
Q1= valor2.quantile(.25)
Q3= valor2.quantile(.75)
IIQ= Q3-Q1
limite_inferior = Q1 - (1.5*IIQ)
limite_superior = Q3 + (1.5*IIQ)

In [ ]:
selecao = (valor2 >= limite_inferior) & (valor2 <= limite_superior)
dados_Price = dados[selecao]

In [ ]:
Box_Price = dados_Price.boxplot(column='Price', figsize=(12,7))

In [ ]:
dados.Price.unique()

# Clustering 
* Devido a grande variedade de preços, vamos clusterizar essa variavel com a biblioteca KMeans
* Vamos usar o DataFrame_new contendo a seleção dos dados minimos instalados, pois essa base não consta mais os outliers.

In [ ]:
dados_new = dados_Minimum_Installs

In [ ]:
dados_new.head()

In [ ]:
Box_Category = dados_new.boxplot(['Rating'],by=['Category'], figsize=(80,7))

In [ ]:
modelo = KMeans(n_clusters = 10, init = 'random', random_state=0)

In [ ]:
X=dados_new[['Price']]

In [ ]:
X

In [ ]:
price = modelo.fit(X)

In [ ]:
price.cluster_centers_

# Investigando
Após clusterizar os dados pelo preço comercializado, notamos uma grande discrepancia entre as classes. Vamos investigar um pouco mais a funco esse detalhe.

In [ ]:
label = price.cluster_centers_

In [ ]:
dados_new.Currency.unique()

# Discrepancia encontrada na base
Em nosso banco de dados temos 54 tipos difernetes de moedas, causando uma discrepancia muito elevada em nossa análise. Teremos de fazer um tratamento discartando os dados com menos frequencia.Para isso vamos primeiramente verificar quais moedas possuem maior influencia em nosso banco de dados.

In [ ]:
dados_new.groupby(by=['Currency']).count()[['Price']]

In [ ]:
dados_new.groupby(by=['Currency']).sum()

# Tratando a discrepancia
Análisando os dados a classificação 'USD' e 'XXX' possuem maior quantidade de index em nossa base. 
Vamos deixar em nossa base nova apenas essas duas variáveis

In [ ]:
dados_new.shape

In [ ]:
dados_new.head()

# Consulta na base
Fazendo consulta na base de dados usando query

In [ ]:
dados_filtrados = dados_new.query('Currency =="USD" or Currency == "XXX"')

In [ ]:
dados_filtrados.head()

In [ ]:
plt.figure( figsize=(80, 5))
ax = plt.hist(data=dados_filtrados, x='Category')

In [ ]:
quant_cat=pd.DataFrame(dados_filtrados.Category.value_counts()).iloc[:10]
quant_cat

In [ ]:
quant_cat.rename(columns={'Category':'Count'}, inplace=True)
quant_cat

In [ ]:
quant_cat.index.name ='Category' 
quant_cat

# Gerando gráficos

In [ ]:
fig, ax = plt.subplots(figsize=(15,6))
sns.barplot(data=quant_cat, x=quant_cat.index, y=quant_cat['Count'], ax=ax, palette=[ '#0C154A','#0C154A','#0C154A','#111E6C','#111E6C','#111E6C','#192DA1','#192DA1','#2039CC','#2039CC'])
#adicionando título
ax.set_title("Quantidade x Categoria", fontdict={'fontsize':15})

#mudando nome e tamanho do label x
ax.set_xlabel('Categoria', fontdict={'fontsize':13})

#mudando nome e tamanho do label y
ax.set_ylabel('Quantidades', fontdict={'fontsize':14})

#mudando tamanho dos labels dos ticks
ax.tick_params(labelsize=10)


#otimizar espaço da figure
fig.tight_layout();

In [ ]:
quant_class=dados_filtrados.groupby(['Category']).mean()[['Rating']].sort_values(by='Rating',ascending=False).iloc[:10]
quant_class

In [ ]:
fig, ax2 = plt.subplots(figsize=(15,6))
sns.barplot(data=quant_class, x=quant_class.index, y=quant_class['Rating'], ax=ax2, palette=[ '#735702','#735702','#735702','#A68B03','#A68B03','#A68B03','#D9B504','#D9B504','#D9CB04','#D9CB04'])
#adicionando título
ax2.set_title("Quantidade de Estrelas x Categoria", fontdict={'fontsize':15})

#mudando nome e tamanho do label x
ax2.set_xlabel('Categoria', fontdict={'fontsize':13})

#mudando nome e tamanho do label y
ax2.set_ylabel('Quantidades de Estrelas', fontdict={'fontsize':14})

#mudando tamanho dos labels dos ticks
ax2.tick_params(labelsize=10)


#otimizar espaço da figure
fig.tight_layout();

# Tabela dos dados agrupados

In [ ]:
agrupado = dados_filtrados.groupby(['Category']).mean().sort_values(by='Maximum_Installs',ascending=False).iloc[:20]
agrupado

In [ ]:
agrupado_min = dados_filtrados.groupby(['Category']).mean().sort_values(by='Maximum_Installs',ascending=False).iloc[-10:]
agrupado_min

In [ ]:
fig, (g1,g2) = plt.subplots(nrows=2, ncols=1, figsize=(30,6))

sns.barplot(ax=g1,data=agrupado, x=agrupado.index, y=agrupado['Maximum_Installs'], palette=[ '#0C154A','#0C154A','#0C154A','#111E6C','#111E6C','#111E6C','#192DA1','#192DA1','#2039CC','#2039CC'])
sns.barplot(ax=g2,data=agrupado, x=agrupado.index, y=agrupado['Rating'], palette=[ '#735702','#735702','#735702','#A68B03','#A68B03','#A68B03','#D9B504','#D9B504','#D9CB04','#D9CB04'])

#adicionando título
g1.set_title("Quantidade de Instalação_Maxima (Média) x Categoria", fontdict={'fontsize':15})
g2.set_title("Quantidade de Estrelas (Média) x Categoria", fontdict={'fontsize':15})

#mudando nome e tamanho do label x
g1.set_xlabel('Categoria', fontdict={'fontsize':13})
g2.set_xlabel('Categoria', fontdict={'fontsize':13})

#mudando nome e tamanho do label y
g1.set_ylabel('Qt_Instalação (Média)', fontdict={'fontsize':14})
g2.set_ylabel('Qt_Estrelas(Média)', fontdict={'fontsize':14})

#mudando tamanho dos labels dos ticks
g1.tick_params(labelsize=10)
g2.tick_params(labelsize=10)

#otimizar espaço da figure
fig.tight_layout();


In [ ]:
fig, (g1,g2) = plt.subplots(nrows=2, ncols=1, figsize=(15,6))

sns.barplot(ax=g1,data=agrupado_min, x=agrupado_min.index, y=agrupado_min['Maximum_Installs'], palette=[ '#0C154A','#0C154A','#0C154A','#111E6C','#111E6C','#111E6C','#192DA1','#192DA1','#2039CC','#2039CC'])
sns.barplot(ax=g2,data=agrupado_min, x=agrupado_min.index, y=agrupado_min['Rating'], palette=[ '#735702','#735702','#735702','#A68B03','#A68B03','#A68B03','#D9B504','#D9B504','#D9CB04','#D9CB04'])

#adicionando título
g1.set_title("Quantidade de Instalação_Minima (Média) x Categoria", fontdict={'fontsize':15})
g2.set_title("Quantidade de Estrelas (Média) x Categoria", fontdict={'fontsize':15})

#mudando nome e tamanho do label x
g1.set_xlabel('Categoria', fontdict={'fontsize':13})
g2.set_xlabel('Categoria', fontdict={'fontsize':13})

#mudando nome e tamanho do label y
g1.set_ylabel('Qt_Instalação (Média)', fontdict={'fontsize':14})
g2.set_ylabel('Qt_Estrelas (Média)', fontdict={'fontsize':14})

#mudando tamanho dos labels dos ticks
g1.tick_params(labelsize=10)
g2.tick_params(labelsize=10)

#otimizar espaço da figure
fig.tight_layout();

In [ ]:
comics_min = 3973 * 0.0927
comics_max = 8858 * 0.0927

comics_media =(comics_max+ comics_min)/2 

simulation_min = 4413 * 0.2137
simulation_max = 10134 * 0.2137

simulation_media =(simulation_max + simulation_min)/2 

print('\nCalculo de Potencial Lucro\n')
print(f'comics_min= {comics_min} | comics_max= {comics_max} | comics_media= {comics_media}')
print(f'\nsimulation_min= {simulation_min} | simulation_max= {simulation_max} | simulation_media= {simulation_media}')
print(f'\nsimulation_min= {round((simulation_min/comics_min)*100,1)}% | simulation_max= {round((simulation_max/comics_max)*100,1)}% | simulation_media= {round((simulation_media/comics_media)*100,1)}%')


# Criando Tabelas

In [ ]:

potencial_lucro = pd.DataFrame(index=['Comics','Simulation','Simulation em %'], data= {'Minimo_Lucro':[368,943,256.1],
                                                     'Maximo_Lucro':[821,2165,263.7],'Media_Lucro':[594,1554,261.4]})
potencial_lucro

In [ ]:
tb_lucro=pd.DataFrame(data={'Lucro_maximo':round(agrupado['Maximum_Installs']*agrupado['Price'],1),'Lucro_minimo':round(agrupado['Minimum_Installs']*agrupado['Price'],1)})
tb_lucro['Lucro_medio']= (tb_lucro['Lucro_maximo']+tb_lucro['Lucro_minimo'])/2

In [ ]:
tb_lucro = tb_lucro.sort_values(by='Lucro_medio', ascending=False)
tb_lucro['Rank_%'] = tb_lucro['Lucro_medio'].rank(pct=True)
tb_lucro['Rank_%_(-1)'] = 1- tb_lucro['Rank_%']
tb_lucro['Class'] = 21-tb_lucro['Lucro_medio'].rank(method='min')
tb_lucro

In [ ]:
print(f'Diferença em % entre as classificações Simulation ')
print(f'\nComics | 1° Categoria')
print(f'\n{ round(( (2.8-2.6)/2.8 ) *100,1)}% | { round(((3.17-2.8)/3.17 ) *100,1) }%')


In [ ]:
diferenca_c_simulation = pd.DataFrame(index=['Comics','1° no Ranking'] ,data={'Dif_Simulation_%':[7.1,11.7]})
diferenca_c_simulation

In [ ]:
aa = round(((4657-4413)/4657)*100,1)
bb = round(((11041-10134)/11041)*100,1)

aa0 = round(((4657-3973)/4657)*100,1)
bb0 = round(((11041-8858)/11041)*100,1)


print(f'Diferença em % entre Crescimento  Simulation ')
print(f'\n1° Categoria')
print(f'\nsimulation_min= {aa}% | simulation_max= {bb}% | simulation_media= {round((bb+aa)/2,1)}%' )



print(f'\n\n\n\nDiferença em % entre Crescimento  Comics ')
print(f'\n1° Categoria')
print(f'\nsimulation_min= {aa0}% | simulation_max= {bb0}% | simulation_media= {round((bb0+aa0)/2,1)}%' )

In [ ]:
crescimento_dif_simulation = pd.DataFrame(index=['dif_simulation','dif_comics'],data={'1°_Rankin_Minimo_%': [5.2,14.7],'1°_Rankin_Maximo_%':[8.2,19.8],'1°_Rankin_Media_%':[6.7,17.2]})
crescimento_dif_simulation

In [ ]:
tb_crescimento = pd.DataFrame(data=agrupado['Maximum_Installs'])
tb_crescimento['Rank_%'] = tb_crescimento['Maximum_Installs'].rank(pct=True)
tb_crescimento['Rank_%_(-1)'] = 1-tb_crescimento['Rank_%']
tb_crescimento['Class'] = 21-tb_crescimento['Maximum_Installs'].rank(method='min')
tb_crescimento

# Conclusão

Análisando os gráficos médios de instalação e classificação detectamos uma boa oportunidade para criação de aplicativos no seguimentos de comics e simulation. Tendo respectivamente: 

> Category   | Rating | Minimum_Installs | Maximum_Installs | Price

>  Comics    |  2.6   |       3973       |      8858        | 0.0927

> Simulation |   2.8  |       4413       |      10134       | 0.2137


Após o cálculo de potencial de Lucro constatamos que o simulation possui maior potencial de lucro. Porem o categoria com maior potencial de crescimento foi o comics.